<a href="https://colab.research.google.com/github/Byrnesz/point-of-interest-data/blob/master/Quickly_Visualize_SafeGraph_Data_On_A_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://global-uploads.webflow.com/5baafc2653bd67278f206724/5be267a03f7813daf821b31e_safegraph-logo-hidpi%403x-p-500.png)

# Quickly Visualize SafeGraph Data On A Map
## Light-weight open source map making with python

--------------
**[Ryan Fox Squire](https://www.linkedin.com/in/ryanfoxsquire/) | Product Data Scientist, [SafeGraph](https://safegraph.com/?utm_source=content&utm_medium=referral&utm_campaign=colabnotebook&utm_content=python_map_viz)**


March 2020

--------------
*Share this notebook: [Shareable Link](https://colab.research.google.com/drive/1_0KvKUMYP1mf6ZAhM0X4LTDUPHzsz40e#offline=true&sandboxMode=true)*



# A Map Is Worth.... More Than One Thousand Words

* When you are trying to understand geospatial data, it *really, really* helps to visualize it on a map.  But how?
* Here I demonstrate a simple python library to make simple easy maps. 
* You can copy this notebook and upload your SafeGraph data directly to make your own maps. 


---

For this demonstration, I will use SafeGraph Core Places and Patterns data about Home Depot in the USA. 

I can use [SafeGraph Core Places](https://docs.safegraph.com/docs#section-core-places?utm_source=content&utm_medium=referral&utm_campaign=colabnotebook&utm_content=python_map_viz) to get centroid locations of each Home Depot and [SafeGraph Patterns](https://docs.safegraph.com/docs/places-schema#section-patterns?utm_source=content&utm_medium=referral&utm_campaign=colabnotebook&utm_content=python_map_viz) data for foot-traffic insights. 



In [0]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import os

from google.colab import drive as mountGoogleDrive 


pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)



# **How to use this notebook**:


---


**Quick Start:** 
1. Read this notebook as a tutorial. **Enjoy!**
2. Do not connect or re-run the cells unless you have your own data (see Customize Analysis, below)

---



In [0]:
your_name = "EDIT_YOUR_NAME_HERE" 

################################
# These commands allow the notebook to read your data from your GoogleDrive
print(f"Hello {your_name}, you will be asked to authenticate for Google File Stream.")
mountGoogleDrive.mount('/content/mountedDrive')
print(f"Congrats {your_name}, you mounted your google drive!")
################################

Hello EDIT_YOUR_NAME_HERE, you will be asked to authenticate for Google File Stream.
Drive already mounted at /content/mountedDrive; to attempt to forcibly remount, call drive.mount("/content/mountedDrive", force_remount=True).
Congrats EDIT_YOUR_NAME_HERE, you mounted your google drive!




**Optional -- Customize Your Analysis**

Want to use your own SafeGraph data?  It's super easy to use this notebook to analyze ***any*** data from SafeGraph.  If not, then just skip these steps and go to the next cell. 

**Step 0.**   Save this Google CoLab notebook to your Google Drive by clicking File > "Save a copy in Drive" in the upper left. 

**Step 1.** Get SafeGraph Data (if you already have data, skip to step 2)
1. Go to [The SafeGraph Data Bar](https://shop.safegraph.com/?utm_source=content&utm_medium=referral&utm_campaign=colabnotebook&utm_content=python_map_viz), and get Core Places and Patterns data.
  1. Select which brand(s) you want to analyze. Maybe you and a top competitor?
  2. Use the coupon code `PythonMapViz` for $200 of free data. Wow! 
  3. Pro-tip: You can filter down to a State (Region), or a city if you want to bring down the cost.
2. Download the data and open them up (after downloading, **unzip** the file by right-click > `Open` or `Unzip`). After unzipping, you will have a directory containing multiple files ([see more info here](https://docs.safegraph.com/docs/getting-started?utm_source=content&utm_medium=referral&utm_campaign=colabnotebook&utm_content=python_map_viz)):
  1. `home_panel_summary.csv` 
  2.  one or more `core_poi-geometry-patterns` csv files 
  3. `visit_panel_summary.csv` 
  4. and a `README.txt`
  5. For complete documentation on these files see [SafeGraph Docs.](https://docs.safegraph.com/docs/places-schema#section-pattern/?utm_source=content&utm_medium=referral&utm_campaign=colabnotebook&utm_content=python_map_viz)

**Step 2.** Copy Data Into Your Google Drive
  1. Now, open your [Google Drive](https://drive.google.com), create a new folder (e.g., `SafeGraph_Data`) and upload the ***unzipped*** SafeGraph files from Step 1 into your new GDrive folder. Easiest is to click-drag the entire folder into your Google Drive. 
  2. Mount your Google Drive to this notebook. You probably already did this by running the above cell. You should see a message printed above that says `Mounted at /content/mountedDrive`. 

**Step 3.** Copy/paste your mounted drive path

1. In the left-side pop-out panel, click the Folder icon for `Files`. Find the folder called `mountedDrive`. Click into `mountedDrive` (be patient, sometimes it lags) and locate the directory into which you uploaded the SafeGraph  data above in **Step 2**. 
2. Right-click on that folder and select `Copy path`. [If this isn't working, then you will have to manually transcribe the path]. Paste the path into the cell below for the variable `sg_data_directory`. 



# Load sample of SafeGraph Patterns & Core Places data: Home Depot locations in the United States


In [0]:
sg_data_directory = "/content/mountedDrive/My Drive/publicly_shared/HomeDepot-CoreGeoPatterns-Jan2020/"
sg_data_files = [sg_data_directory+f for f in os.listdir(sg_data_directory) if ("core_poi" in f.lower()) | ("patterns" in f.lower())]
sg_data = pd.concat([pd.read_csv(f) for f in sg_data_files], sort=False)
print(sg_data.shape)
sg_data.head()

(3968, 38)


,safegraph_place_id,parent_safegraph_place_id,safegraph_brand_ids,location_name,brands,top_category,sub_category,naics_code,latitude,longitude,street_address,city,region,postal_code,open_hours,category_tags,polygon_wkt,polygon_class,phone_number,is_synthetic,includes_parking_lot,iso_country_code,date_range_start,date_range_end,raw_visit_counts,raw_visitor_counts,visits_by_day,visitor_home_cbgs,visitor_work_cbgs,visitor_country_of_origin,distance_from_home,median_dwell,bucketed_dwell_times,related_same_day_brand,related_same_month_brand,popularity_by_hour,popularity_by_day,device_type
0,sg:bfb88a1910434b169e5733a7015eb192,sg:d65e327ffc174e558b1c4450da1b47f2,SG_BRAND_8cebb51fb05f1bf005e53bfec08d6c10,The Home Depot,The Home Depot,Building Material and Supplies Dealers,Hardware Stores,444130,33.988801,-117.704133,14549 Ramona Ave,Chino,CA,91710,"{ ""Mon"": [[""6:00"", ""22:00""]], ""Tue"": [[""6:00"",...",NaN,POLYGON ((-117.7045127749443 33.98971909914596...,OWNED_POLYGON,1.909394e+10,False,NaN,US,1.575158e+09,1.577837e+09,2415.0,1554.0,"[108,83,59,66,90,101,136,114,104,79,77,76,89,9...","{""060710019031"":77,""060710001162"":42,""06071000...","{""060710005043"":33,""060710001152"":32,""06071000...","{""US"":1529}",5272.0,19.0,"{""<5"":53,""5-20"":1277,""21-60"":805,""61-240"":214,...","{""McDonald's"":10}","{""Starbucks"":44,""Target"":38,""Costco Wholesale ...","[4,4,4,4,3,14,46,96,157,241,295,333,351,357,35...","{""Monday"":382,""Tuesday"":301,""Wednesday"":201,""T...","{""android"":957,""ios"":597}"
1,sg:c5f302af4c5b4dfe9c88d4ac20b22f09,sg:cfe9f6e3a75c44d583c59b14817f6c32,SG_BRAND_8cebb51fb05f1bf005e53bfec08d6c10,The Home Depot,The Home Depot,Building Material and Supplies Dealers,Hardware Stores,444130,33.912652,-83.444808,1740 Epps Bridge Pkwy,Athens,GA,30606,"{ ""Mon"": [[""6:00"", ""22:00""]], ""Tue"": [[""6:00"",...",NaN,POLYGON ((-83.44516396522522 33.91324411319755...,OWNED_POLYGON,1.706354e+10,False,NaN,US,1.575158e+09,1.577837e+09,2549.0,1738.0,"[133,73,80,86,60,78,90,105,75,74,65,71,62,122,...","{""132190301002"":91,""132190303001"":59,""13219030...","{""132190301001"":33,""132190301002"":25,""13219030...","{""US"":1712}",18250.0,20.0,"{""<5"":50,""5-20"":1341,""21-60"":830,""61-240"":208,...","{""Lowe's"":14,""Tires Plus"":8,""Academy Sports + ...","{""Walmart"":50,""Chick-fil-A"":47,""Kroger"":41,""Lo...","[11,11,15,13,35,72,88,131,215,285,341,450,443,...","{""Monday"":383,""Tuesday"":358,""Wednesday"":216,""T...","{""android"":860,""ios"":879}"
2,sg:f7468238732b45d9bf7f662efdc342a0,NaN,SG_BRAND_8cebb51fb05f1bf005e53bfec08d6c10,The Home Depot,The Home Depot,Building Material and Supplies Dealers,Hardware Stores,444130,30.350485,-91.028705,18139 Highland Rd,Baton Rouge,LA,70810,"{ ""Mon"": [[""6:00"", ""22:00""]], ""Tue"": [[""6:00"",...",NaN,POLYGON ((-91.02857887744904 30.35129397140938...,OWNED_POLYGON,1.225755e+10,False,NaN,US,1.575158e+09,1.577837e+09,1919.0,1265.0,"[84,59,54,58,51,54,64,111,62,49,59,57,71,86,77...","{""220050302031"":102,""220330045101"":63,""2203300...","{""220330040092"":40,""220050303003"":20,""22005030...","{""US"":1235}",8702.0,19.0,"{""<5"":44,""5-20"":1069,""21-60"":592,""61-240"":132,...","{""Walmart"":7}","{""Walmart"":56,""Shell Oil"":36,""Walgreens"":33,""R...","[8,7,5,4,5,27,55,105,166,220,267,330,342,338,3...","{""Monday"":312,""Tuesday"":278,""Wednesday"":180,""T...","{""android"":516,""ios"":750}"
3,sg:19d8ab8d4ed34c5e89b1ec29371e1057,NaN,SG_BRAND_8cebb51fb05f1bf005e53bfec08d6c10,The Home Depot,The Home Depot,Building Material and Supplies Dealers,Hardware Stores,444130,39.058327,-76.960460,2300 Broadbirch Dr,Silver Spring,MD,20904,"{ ""Mon"": [[""6:00"", ""22:00""]], ""Tue"": [[""6:00"",...",NaN,"POLYGON ((-76.960631 39.05891, -76.960308 39.0...",OWNED_POLYGON,1.301680e+10,False,NaN,US,1.575158e+09,1.577837e+09,1879.0,1161.0,"[61,61,62,60,67,60,87,65,60,64,65,60,59,72,81,...","{""240317014181"":25,""240317014141"":21,""24031701...","{""240317014211"":14,""240317015061"":7,""240

# Use the python library Folium for quick and simple maps

[Folium](https://python-visualization.github.io/folium/) is a nice open-source library for visualizing points on an ***interactive*** map. And it comes pre-installed in Google CoLab!

**Pros:**
* Light-weight, can quickly start exploratory data analysis on a map.
* Easy to learn [Quick Start Guide](https://python-visualization.github.io/folium/quickstart.html)
* Easy to make interactive maps, which are more powerful for exploratory data analysis
* Has some level of customization availble if you dive into the docs

**Cons:**
* Not performant (it is slow) 
* Not scalable (does not handle many thousands of points/features)
* Not built for analytics (no geospatial analytic functions)
* Not much GIS/cartographic control (e.g. no map projections). 


IMO, Folium is a very easy way to go from zero-to-one if you want some simple map visualizations. 

*And sometimes seeing points on a map is extremely powerful.*

Below, I've built a simple wrapper function using folium  to easily visualize many points on a map when your data is organized as one row per point in a pandas DataFrame. 

In [0]:
### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
### Functions for simple mapping using folium 
### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

def make_map_plot(df_, 
                  plot_limit=3000, 
                  radius_col=None, 
                  radius_mod=100, 
                  color='black', 
                  fill_color='red', 
                  fill_opacity = 0.2, 
                  pop_up_col=None,
                  tooltip = 'Click for more info.', 
                  zoom_start=4,
                  tiles = 'OpenStreetMap', 
                  map_width = 700,
                  map_height = 400,
                  marker_type= 'circle', 
                  map_plot=None, 
                  verbose=False):
    # Parameters:
      # df_ is a pandas dataframe. It requires a column called "latitude" and a column called "longitude". 
      # radius_col is a column_name or None. If None, every point is given a fixed radius. 
          # Otherwise, the value in the column radius_col is used as the radius. 
      # radius_mod is to scale your radius units to correspond to units on your map.
      # zoom_start is the scale of the map. Larger numbers = higher resolution.
      # color, fill_color, fill_opacity are marker parameters, see others: https://leafletjs.com/reference-1.3.4.html#path 
      # tiles determines the base layer. Open source options include 'OpenStreetMap', 'Stamen Terrain', 'Stamen Toner'
      # map_width and map_height determine the size of the map image (in pixels)
      # marker_type determines what type of marker is being drawn on the map. Options: 'circle' or 'normal'

    # check valid inputs
    valid_inputs = {'marker_type' : {'val' : marker_type, 'valids' : ['circle', 'normal']},
                    'tiles' :  {'val' : tiles, 'valids': ['OpenStreetMap', 'Stamen Terrain', 'Stamen Toner', 'Mapbox Bright', 'Mapbox Control Room']},
                    'radius_col' : {'val' : radius_col, 'valids' : [None] + [col for col in df_.columns if pd.api.types.is_numeric_dtype(df_[col])]}}
    for param, param_valid_dict in valid_inputs.items():
      if(param_valid_dict['val'] not in param_valid_dict['valids']):
        raise Exception("Invalid parameter input for '{0}'. Valid options are {1}. input value was '{2}' .".format(param, param_valid_dict['valids'], param_valid_dict['val']))   

    # create basemap
    if(not map_plot):
      map_plot = folium.Map(width=map_width,
                            height=map_height,
                            location=[df_.latitude.mean(), df_.longitude.mean()],
                            tiles=tiles,
                            zoom_start=zoom_start,
                            control_scale = True)

    # add markers
    counter = 0
    for index, row in df_.iterrows():
        counter+=1
        if(marker_type=='circle'):
          add_circle_marker_to_map(map_plot, row, radius_col, radius_mod, color, fill_color, fill_opacity, pop_up_col, tooltip)
        elif(marker_type=='normal'):
          add_marker_to_map(map_plot, row, pop_up_col, tooltip)
        if(counter>plot_limit): break
    if(verbose): print("Plotted {0} locations".format(counter))
    return(map_plot)


def add_marker_to_map(map_plot, row, pop_up_col, tooltip):
  folium.Marker([row.latitude, row.longitude],
                        popup= row[pop_up_col] if pop_up_col else None,
                        tooltip=tooltip if pop_up_col else None,
                   ).add_to(map_plot)
  return(None)


def add_circle_marker_to_map(map_plot, row, radius_col, radius_mod, color, fill_color, fill_opacity, pop_up_col, tooltip):
  folium.CircleMarker([row.latitude, row.longitude],
                        radius= row[radius_col]/radius_mod if radius_col else 2,
                        color = color,
                        fill_color = fill_color,
                        weight=0.5,
                        fill_opacity= fill_opacity,
                        popup= row[pop_up_col] if pop_up_col else None,
                        tooltip=tooltip if pop_up_col else None,
                   ).add_to(map_plot)
  return(None)


# Example # 1: Visualize Points On A Map

In [0]:
print("Figure 1: A subset of Home Depot Locations in the USA. Simple points on a map")
data2map_df = sg_data.sample(2000)
make_map_plot(data2map_df, fill_color='red', fill_opacity=1)

Figure 1: A subset of Home Depot Locations in the USA. Simple points on a map


# Example #2: Visualize Points On A Map With Interactive Pop Up Info

Note use of `zoom_start` parameter to adjust default map.


And use of `pop_up_col` to control what info is displayed when user clicks on marker. 


Also to illustrate more capabilities, I use a different basemap. 

In [0]:
print("Figure 2: Home Depot locations in Ohio. \n\nClick a marker to see pop-up info\n")
data2map_df = sg_data[sg_data.region =='OH'].copy()
data2map_df['pop_up_col'] = data2map_df.location_name + '\n' + data2map_df.street_address+ '\n' + data2map_df.city + '\n' + data2map_df.region
make_map_plot(data2map_df, 
              zoom_start=7, 
              marker_type = 'normal',
              pop_up_col = 'pop_up_col',
              tiles='Stamen Toner')

Figure 2: Home Depot locations in Ohio. 

Click the marker to see pop-up info



# Example #3: Use Radius To Visualize Extra Information

Often we want to visualize a 3rd dimension besides just latitude and longitude. One way to do that is to use the radius of the point to communicate more information on your map. 

Set `marker_type` = `circle`

And use the parameter `radius_col` to specify which column should be plotted as the radius. 


In [0]:
print("Figure 3: Foot-traffic of locations in Ohio.")
print("This quickly shows that the highest foot-traffic location is in Northern Ohio \nlocated at '4330 Leavitt Rd, Lorain, OH' ")
make_map_plot(data2map_df, 
              zoom_start=7, 
              radius_col = 'raw_visitor_counts',
              marker_type = 'circle',
              fill_color = 'yellow',
              pop_up_col = 'pop_up_col',
              tiles='Stamen Toner')

Figure 3: Foot-traffic of locations in Ohio.
This quickly shows that the highest foot-traffic location is in Northern Ohio 
located at '4330 Leavitt Rd, Lorain, OH' 


# Example #4: Visualize Points On A Map Using Different Colors and Styles

You can also use the `make_map_plot()` function *recursively* to add different sets of points with different colors/styles to the same map. 

For example, in Figure 4, below, we color-code locations by foot-traffic. 

----

In [0]:
print("Figure 4: Home Depots in Ohio, color-coded by foot-traffic. \nBlue = Top 20% highest foot-traffic\nRed = Lowest 20% foot-traffic\nGrey = Middling foot-traffic ")

data2map_df = data2map_df.sort_values(by='raw_visitor_counts', ascending=False)
tenthpcttile = int(data2map_df.shape[0]*.2)
data2map_list = [('blue', data2map_df.iloc[0:tenthpcttile] ), 
                 ('red',  data2map_df.iloc[-tenthpcttile::]), 
                 ('grey',  data2map_df.iloc[tenthpcttile:-tenthpcttile])
                 ]
my_map = None
for fill_color, data2map_ in data2map_list:
  my_map = make_map_plot(data2map_, 
                         zoom_start=6, 
                         radius_col = 'raw_visitor_counts',
                         marker_type = 'circle',
                         fill_color = fill_color,
                         pop_up_col = 'pop_up_col',
                         tiles='Stamen Terrain',
                         map_plot = my_map)

my_map

Figure 4: Home Depots in Ohio, color-coded by foot-traffic. 
Blue = Top 20% highest foot-traffic
Red = Lowest 20% foot-traffic
Grey = Middling foot-traffic 


#  Geospatial Data Need Maps

These maps can instantly reveal interesting insights that are hard to uncover without a map.  

For example, in Figure 4, most of the high traffic Home Depot locations (blue) are located in either North East, Central, or South West Ohio (these clusters correspond to the 3 largest cities in Ohio: Cleveland, Columbus, and Cincinatti). 

Interestingly, some of the lowest-traffic locations (red)overlap geographically with the highest traffic locations in the North East (but this is not the case in the South West or in Columbus in central Ohio). 

What does this say about the Home Depot market in North East Ohio? 

# Summary


*   Folium is a useful light-weight library for doing simple map visualizations. 
*   Here we define some simple wrapper functions that make it super easy to build map visualizations from pandas DataFrames.
* If a picture is worth 1000 words, a good map is worth even more, so let's make some maps! 


## Want to make even more powerful maps? 
* There are excellent tools for making more powerful maps with SafeGraph data. 
* First, you can dive deeper into [Folium](https://python-visualization.github.io/folium/). [This is a good article showing how to build maps with custom geometries](https://towardsdatascience.com/making-3-easy-maps-with-python-fb7dfb1036) (and not just points). 
* There are good open source tools, such as [Kepler.gl ](https:/kepler.gl/) and [QGIS](https://www.qgis.org/en/site/). 
* There are also excellent non-open source geospatial analytics and map-making tools.  These include [Esri](https://marketplace.arcgis.com/listing.html?id=3425348e4bee4059af2b353e52df43c2), [Carto](https://go.carto.com/webinars/safegraph-market-analysis-recorded), [Mapbox](https://www.mapbox.com/), [OmniSci](https://www.omnisci.com/), and the [Google Maps API](https://developers.google.com/maps/documentation). SafeGraph data is integrated with many of these solutions. These software platforms offer much more than "just" map making, and can enable complex geospatial analytics and geospatial data science.
* [Tableau](https://help.tableau.com/current/pro/desktop/en-us/maps_howto_simple.htm) and other data analytics platforms also have geospatial visualization capabilities. 





---

*Thanks for reading! If you found this useful or interesting please upvote and share with a friend.* 

*You are strongly encouraged to try out a sample of ***SafeGraph Core Places and Patterns*** data for free, no strings attached at the [SafeGraph Data Bar](https://shop.safegraph.com/?utm_source=content&utm_medium=referral&utm_campaign=colabnotebook&utm_content=python_map_viz).*  *Use coupon code* **`PythonMapViz`** *for $200 worth of free data!*

---

## Contact: 
* Please send us your ideas, feedback, bug discoveries, and suggestions: datastories@safegraph.com
* Or leave us a comment on the blog

<br>
<br>
<br>


![alt text](https://global-uploads.webflow.com/5baafc2653bd67278f206724/5be267a03f7813daf821b31e_safegraph-logo-hidpi%403x-p-500.png)